In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from azureml.core import Experiment
from azureml.core import Model
from azureml.core import Workspace

In [2]:
import azureml.core
print(f'AZURE ML SDK VERSION: {azureml.core.VERSION}')

AZURE ML SDK VERSION: 1.30.0


## Initialize workspace

In [3]:
#connect to the workspace and get model and scaler
ws = Workspace.from_config()
print(ws)

Workspace.create(name='mlops_ws', subscription_id='6f507867-3280-4f79-8820-9939bef45222', resource_group='learn_mlops')


## Deploy model on ACI(Azure Container Instance)

In [44]:
%%writefile score.py
import pickle
import numpy as np
import json
import time
import joblib
import os
import onnxruntime
from azureml.core.model import Model
from azureml.monitoring import ModelDataCollector
from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType

def init():
    global scaler, model, input_name, label_name, inputs_dc, output_dc
    #scaler_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'scaler.pkl')
    scaler_path = Model.get_model_path(model_name='Scaler', version=1)
    scaler = joblib.load(scaler_path)
    
    #model_onnx = os.path.join(os.getenv('AZUREML_MODEL_DIR'), version, 'final_model_svm.onnx')
    model_onnx = Model.get_model_path(model_name='Support-vector-Classifier', version=1)
    model = onnxruntime.InferenceSession(model_onnx, None)
    input_name = model.get_inputs()[0].name
    label_name = model.get_outputs()[0].name
    
    # Variable to monitor model input and output data
    inputs_dc = ModelDataCollector("Support vector classifier model", designation="inputs", feature_names=["feat1", "feat2", "feat3", "feat4", "feat5", "feat6"])
    output_dc = ModelDataCollector("Support vector classifier model", designation="output", feature_names=["weatherprediction"])
# @input_schema("data", NumpyParameterType(np.array([34.927778, 0.24, 7.3899, 83, 16.1000, 1])))
# @output_schema(NumpyParameterType(np.array([0])))

def run(raw_data):
    try:
        #Scaling data
        data = np.array(json.loads(raw_data)['data'])
        data = scaler.transform(data.reshape(1, -1))
        #Infer model
        result = model.run([label_name], {input_name:data.astype(np.float32)})[0]
        result = {'result': np.array(result).tolist()}
        return result
        output_dc.collect(result)
    except Exception as e:
        result = {'result':str(e)}
        return result
        output_dc.collect(result)

Overwriting score.py


#### Define Environnement

In [45]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment(name="my_env")
#env = Environment.get(workspace=ws, name="AzureML-onnxruntime-1.6-ubuntu18.04-py37-cpu-inference").clone("my_env")

In [46]:
for pip_package in ["numpy", "onnxruntime", "joblib", "azureml-core", "azureml-monitoring", "azureml-defaults", 
                    "scikit-learn==0.22.2", "inference-schema", "inference-schema[numpy-support]"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score.py', environment=env)

In [47]:
env

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210513.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "my_env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
       

##### Deployment configuration

In [48]:
from azureml.core.webservice import AciWebservice

deploy_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, collect_model_data=True)
deploy_config

#### deploy web service

In [49]:
model1 = Model(workspace=ws, name="Scaler")
model2 = Model(workspace=ws, name="Support-vector-Classifier")

service_name="weather-aci-prediction"

In [50]:
service = Model.deploy(ws, service_name, models=[model1, model2], inference_config=inference_config, deployment_config=deploy_config,
                      overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-07-15 00:48:13+00:00 Creating Container Registry if not exists.
2021-07-15 00:48:13+00:00 Registering the environment.
2021-07-15 00:48:15+00:00 Use the existing image.
2021-07-15 00:48:15+00:00 Generating deployment configuration.
2021-07-15 00:48:16+00:00 Submitting deployment to compute..
2021-07-15 00:48:20+00:00 Checking the status of deployment weather-aci-prediction..
2021-07-15 00:50:47+00:00 Checking the status of inference endpoint weather-aci-prediction.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [51]:
print(service.get_logs())

2021-07-15T00:50:33,092511600+00:00 - rsyslog/run 
2021-07-15T00:50:33,095721400+00:00 - gunicorn/run 
File not found: /var/azureml-app/.
Starting HTTP server
2021-07-15T00:50:33,094511600+00:00 - iot-server/run 
2021-07-15T00:50:33,125097800+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-07-15T00:50:33,698534400+00:00 - iot-server/finish 1 0
2021-07-15T00:50:33,702569600+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (63)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 92
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-07-15 00:50:37,317 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2021-07-15 00:50:37,318 | root | INFO | Starting up request id generator
2021-07-15 00:50:37,318 | root | INFO | Starting up app in

In [52]:
print(service.state)

Healthy


#### Test web service

In [53]:
import json

input_payload = json.dumps({
    'data':[34.927778, 0.24, 7.3899, 83, 16.1000, 1]
})
#input_payload = json.dumps({'data':[[34.927778, 0.24, 7.3899, 83, 16.1000, 1]]})
#input_payload = np.array(json.loads(input_payload)['data'][0])
output = service.run(input_payload)

print(output)

{'result': [1]}
